In [2]:
import json
import os
import time
import pandas as pd

In [ ]:
from openai import OpenAI
client = OpenAI()
api_key = "api_key"

### **Generating open ai files for batch processing**

In [ ]:
prompts = []
dfs = []

In [ ]:
for idx, df in enumerate(dfs):
    comments = df["Texto Comentario"].tolist()
    
    for prompt_idx, prompt in enumerate(prompts):
        # Nombre del archivo de salida
        input_filename = f'data/batches/prompt{prompt_idx + 1}/input_comments_batch_{idx + 1}.jsonl'
        
        # Crear directorio si no existe
        os.makedirs(os.path.dirname(input_filename), exist_ok=True)
        
        # Abrir y escribir en el archivo JSONL con codificación UTF-8
        with open(input_filename, 'w', encoding='utf-8') as jsonl_file:
            for i, comment in enumerate(comments, start=1):
                request_data = {
                    "custom_id": f"request-{i}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": "gpt-4o-mini",
                        "temperature": 0,
                        "messages": [
                            {"role": "system", "content": prompt},
                            {"role": "user", "content": comment}
                        ],
                        "max_tokens": 1000
                    }
                }
                jsonl_file.write(json.dumps(request_data, ensure_ascii=False) + '\n')
        
        print(f"Archivo JSONL creado para el lote {idx + 1} y el prompt {prompt_idx + 1}.")

print("Archivos JSONL creados correctamente.")

### **Response processing function**

In [ ]:
def procesar_respuestas(file_response):
    # Leer el contenido del archivo como texto
    file_content = file_response.text
    
    # Convertir cada línea del archivo JSONL en un diccionario
    lines = file_content.splitlines()
    responses = [json.loads(line) for line in lines]
    
    # Extraer la respuesta del asistente
    assistant_responses = [
        response['response']['body']['choices'][0]['message']['content'] 
        for response in responses
    ]
    
    # Crear un DataFrame con las respuestas en una columna
    df_respuestas = pd.DataFrame(assistant_responses, columns=['respuesta'])
    
    # Mostrar el DataFrame
    return df_respuestas

### **Run batch per prompts an index**

In [ ]:
# Suponiendo que 'prompts' es una lista de los textos de los prompts
prompt = prompts[0]  # Selecciona el primer prompt
prompt_idx = 5  # Índice del primer prompt

# Establecer el índice de inicio, establecer a 0 para procesar todos desde el principio
start_index = 5

# Usar 'start_index' para definir el inicio del bucle
for idx, df in enumerate(dfs[start_index:], start=start_index):

    # Nombre del archivo de entrada basado en el primer prompt y el lote
    input_filename = f'data/batches/prompt{prompt_idx + 1}/input_comments_batch_{idx + 1}.jsonl'
    
    # Subir el archivo a OpenAI
    batch_input_file = client.files.create(
        file=open(input_filename, "rb"),
        purpose="batch"
    )
    batch_input_file_id = batch_input_file.id

    # Crear un lote de procesamiento
    batch_process = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={"description": f"prompt{prompt_idx + 1}.final"}
    )
    batch_id = batch_process.id

    print(f"Lote {idx + 1} y prompt {prompt_idx + 1} creado.")
    print(f"ID del lote: {batch_id}")

    # Esperar a que el lote se confirme que está en proceso
    while True:
        status = client.batches.retrieve(batch_id).status
        if status == "processing":
            print(f"El lote {idx + 1} y el prompt {prompt_idx + 1} está en proceso.")
            break
        elif status not in ["queued", "processing"]:
            print(f"El lote {idx + 1} y el prompt {prompt_idx + 1} ha fallado o sido cancelado en la etapa de inicio.")
            break
        time.sleep(10)  # Espera 10 segundos antes de comprobar de nuevo

    # Esperar a que el lote se complete
    while True:
        status = client.batches.retrieve(batch_id).status
        if status == "completed":
            # si el lote se ha completado, procesar las respuestas
            file_response_t = client.batches.retrieve(batch_id).output_file_id  # se obtiene el id del archivo de salida
            file_response = client.files.content(file_response_t)
            df_respuestas = procesar_respuestas(file_response)  # Asegúrate de que esta función retorne un DataFrame
            print(f"Respuestas procesadas para el lote {idx + 1} y el prompt {prompt_idx + 1}.")
            df_respuestas.to_csv(f'data/batches/prompt{prompt_idx + 1}/results2/batch_{idx + 1}_with_responses.csv', index=False)
            break
        elif status in ["failed", "cancelled"]:
            print(f"El lote {idx + 1} y el prompt {prompt_idx + 1} ha fallado o sido cancelado.")
            break
        time.sleep(100)  # Espera 100 segundos antes de comprobar de nuevo

print("Procesamiento de respuestas completado.")

### **All prompts processing fuction**

In [ ]:
# Suponiendo que 'prompts' es una lista de los textos de los prompts
start_index = 0  # Establecer el índice de inicio para los DataFrame

# Iniciar iteración sobre los prompts desde el índice 1 (que corresponde al segundo prompt)
for prompt_idx, prompt in enumerate(prompts[6:], start=6): 
    
    # Iterar sobre cada DataFrame a partir del índice de inicio
    for idx, df in enumerate(dfs[start_index:], start=start_index):
        
        # Nombre del archivo de entrada basado en el prompt y el lote
        input_filename = f'data/batches/prompt{prompt_idx + 1}/input_comments_batch_{idx + 1}.jsonl'
        
        # Crear la carpeta 'results2' si no existe
        results_dir = f'data/batches/prompt{prompt_idx + 1}/results2'
        os.makedirs(results_dir, exist_ok=True)
        
        # Subir el archivo a OpenAI
        batch_input_file = client.files.create(
            file=open(input_filename, "rb"),
            purpose="batch"
        )
        batch_input_file_id = batch_input_file.id

        # Crear un lote de procesamiento
        batch_process = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={"description": f"prompt{prompt_idx + 1}.final"}
        )
        batch_id = batch_process.id

        print(f"Lote {idx + 1} y prompt {prompt_idx + 1} creado.")
        print(f"ID del lote: {batch_id}")

        # Esperar a que el lote se confirme que está en proceso
        while True:
            status = client.batches.retrieve(batch_id).status
            if status == "processing":
                print(f"El lote {idx + 1} y el prompt {prompt_idx + 1} está en proceso.")
                break
            elif status not in ["queued", "processing"]:
                print(f"El lote {idx + 1} y el prompt {prompt_idx + 1} ha fallado o sido cancelado en la etapa de inicio.")
                break
            time.sleep(10)  # Espera 10 segundos antes de comprobar de nuevo

        # Esperar a que el lote se complete
        while True:
            status = client.batches.retrieve(batch_id).status
            if status == "completed":
                # si el lote se ha completado, procesar las respuestas
                file_response_t = client.batches.retrieve(batch_id).output_file_id  # se obtiene el id del archivo de salida
                file_response = client.files.content(file_response_t)
                df_respuestas = procesar_respuestas(file_response)  # Asegúrate de que esta función retorne un DataFrame
                print(f"Respuestas procesadas para el lote {idx + 1} y el prompt {prompt_idx + 1}.")
                
                # Guardar las respuestas en la carpeta 'results2'
                df_respuestas.to_csv(f'{results_dir}/batch_{idx + 1}_with_responses.csv', index=False)
                break
            elif status in ["failed", "cancelled"]:
                print(f"El lote {idx + 1} y el prompt {prompt_idx + 1} ha fallado o sido cancelado.")
                break
            time.sleep(100)  # Espera 100 segundos antes de comprobar de nuevo

print("Procesamiento de respuestas completado.")